# data

In [1]:
from os import listdir
from os.path import isfile, join
easy_ham = [f for f in listdir("data/easy_ham") if isfile(join("data/easy_ham", f))] # 2500 = 1716+784
easy_ham_2 = [f for f in listdir("data/easy_ham_2") if isfile(join("data/easy_ham_2", f))] # 1400 = 960+440
hard_ham = [f for f in listdir("data/hard_ham") if isfile(join("data/hard_ham", f))] # 250 = 172+78
spam = [f for f in listdir("data/spam") if isfile(join("data/spam", f))] # 500 = 344+156
spam_2 = [f for f in listdir("data/spam_2") if isfile(join("data/spam_2", f))] # 1396 = 958+438
for index in range(len(easy_ham)):
    easy_ham[index] = "data/easy_ham/"+ easy_ham[index]
for index in range(len(easy_ham_2)):
    easy_ham_2[index] = "data/easy_ham_2/"+ easy_ham_2[index]
for index in range(len(hard_ham)):
    hard_ham[index] = "data/hard_ham/"+ hard_ham[index]
for index in range(len(spam)):
    spam[index] = "data/spam/"+ spam[index]
for index in range(len(spam_2)):
    spam_2[index] = "data/spam_2/"+ spam_2[index]

In [2]:
easy_ham_2

['data/easy_ham_2/00001.1a31cc283af0060967a233d26548a6ce',
 'data/easy_ham_2/00002.5a587ae61666c5aa097c8e866aedcc59',
 'data/easy_ham_2/00003.19be8acd739ad589cd00d8425bac7115',
 'data/easy_ham_2/00004.b2ed6c3c62bbdfab7683d60e214d1445',
 'data/easy_ham_2/00005.07b9d4aa9e6c596440295a5170111392',
 'data/easy_ham_2/00006.654c4ec7c059531accf388a807064363',
 'data/easy_ham_2/00007.2e086b13730b68a21ee715db145522b9',
 'data/easy_ham_2/00008.6b73027e1e56131377941ff1db17ff12',
 'data/easy_ham_2/00009.13c349859b09264fa131872ed4fb6e4e',
 'data/easy_ham_2/00010.d1b4dbbad797c5c0537c5a0670c373fd',
 'data/easy_ham_2/00011.bc1aa4dca14300a8eec8b7658e568f29',
 'data/easy_ham_2/00012.3c1ff7380f10a806321027fc0ad09560',
 'data/easy_ham_2/00013.245fc5b9e5719b033d5d740c51af92e0',
 'data/easy_ham_2/00014.8e21078a89bd9c57255d302f346551e8',
 'data/easy_ham_2/00015.d5c8f360cf052b222819718165db24c6',
 'data/easy_ham_2/00016.bc1f434b566619637a0de033cd3380d1',
 'data/easy_ham_2/00017.8b965080dfffada165a54c041c27e33f

In [3]:
# spam rate
(500+1396)/(2500+1400+250+500+1396)

0.3135957657955673

In [4]:
2500+1400+250+500+1396

6046

In [5]:
0.3135957657955673*2500

783.9894144889182

In [6]:
type(spam)

list

In [7]:
list1 = ['a','b','c']
list2 = ['A','B','C']

In [8]:
type(list1)

list

In [9]:
list3 = list1[0:2]

In [10]:
list3

['a', 'b']

In [11]:
list3 += list2[0:2]

In [12]:
list3

['a', 'b', 'A', 'B']

In [13]:
Training_Ham = easy_ham[0:1716]
Training_Ham += easy_ham_2[0:960]
Training_Ham += hard_ham[0:172]
len(Training_Ham)

2848

In [14]:
Testing_Ham = easy_ham[1716+1:]
Testing_Ham += easy_ham_2[960+1:]
Testing_Ham += hard_ham[172+1:]
len(Testing_Ham)

1299

In [15]:
Training_Spam = spam[0:344]
Training_Spam += spam_2[0:958]
len(Training_Spam)

1302

In [16]:
Testing_Spam = spam[344+1:]
Testing_Spam += spam_2[958+1:]
len(Testing_Spam)

592

In [17]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
s=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
import re
import codecs
frequency_list = [1, 2, 3, 5, 10, 20, 30, 50, 100, 200, 300, 500, 1000, 2000, 10000]
# frequency_list = [1000]

dictionary = dict()
Training = Training_Ham + Training_Spam
Testing = Testing_Ham + Testing_Spam
f = open("ret_from/NN.txt", "a")

In [19]:
from sklearn.metrics import classification_report
def eva_fun_2 (real, predict, algo):
    if (len(real) != len(predict)):
        print("Error: length is not the same")
        return -1
#     real = real.astype('int')
#     predict = predict.astype('int')
    target_names = ['ham', 'Spam']
    print("Algorithm: " + algo)
    f.write("Algorithm: " + algo + '\n')
    print(classification_report(real, predict, target_names=target_names))
    f.write(classification_report(real, predict, target_names=target_names))
    f.write('\n')

In [20]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

for frequency in frequency_list:
    print('frequency is ' + str(frequency))
    f.write('\nfrequency is ' + str(frequency) + '\n')
    dictionary = dict()
    
    for file_names in Training:
        #f = open(file_names, "r")
        ## UnicodeDecodeError: 'cp950' codec can't decode byte 0x93 in position 1885: illegal multibyte sequence
        #f = open(file_names, "r", encoding="utf-8")
        ## UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 1885: invalid start byte

        file = codecs.open(file_names, 'r', encoding='utf-8', errors='ignore')

        body = file.read().split("\n\n", 1)[1].lower()
        #word = re.findall(r"[\w']+", body) # 93783 ## 21525
        word = re.findall(r"[a-zA-Z']+", body) # 123656 ## 21224
        for i in word:
            dictionary[i] = dictionary.get(i, 0) + 1
#     print(len(dictionary))
    dictionary = {i:dictionary[i] for i in dictionary if dictionary[i]>frequency}
#     print(len(dictionary))
    #print(dictionary)
    list_dictionary = list(dictionary)
    list_dictionary = list(filter(lambda w: not w in s,list_dictionary))
    list_dictionary
#     print(len(list_dictionary))
    
    # Remove common word in dictionary
    for s__ in s:
        if s__ in dictionary:
            del dictionary[s__]
#     print(len(dictionary))

    print('dict length: ' + str(len(dictionary)))
    f.write('dict length: ' + str(len(dictionary)))
    count = 0
    for i in dictionary:
    #     print(i)
        dictionary[i] = count
        count += 1
    dictionary

    train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = int)
    train_ndarray.shape

    label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
    label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)

    count = 0
    for Ham_mail in Training_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Training_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                train_ndarray[count][dictionary[w]] = 1
        count += 1
    train_dataframe = pd.DataFrame(train_ndarray)

    test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = int)
    test_ndarray.shape

    count = 0
    for Ham_mail in Testing_Ham:
        file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
    #     print(train_ndarray[count])
        count += 1
    for Spam_mail in Testing_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        body = file.read().split("\n\n", 1)[1].lower()
        word = re.findall(r"[a-zA-Z']+", body)
        for w in word:
            if w in dictionary:
                test_ndarray[count][dictionary[w]] = 1
        count += 1
    test_dataframe = pd.DataFrame(test_ndarray)

    NN = MLPClassifier()
    NN.fit(train_dataframe, label_train)

    result = NN.predict(test_dataframe)

    
    
    
    #-------------- from block -----------------
    
    Spam_from = dict()
    count = 0
    for Spam_mail in Training_Spam:
        file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
        word = re.findall(r"From.*\n", file.read())[0]
        word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
        if not word_2 :
            continue
        Spam_from[word_2[0]] = Spam_from.get(i, 0)    
    
    index = 0
    cheat_result = result.copy()
    for mail in Testing:
        file = codecs.open(mail, 'r', encoding='utf-8', errors='ignore')
        word = re.findall(r"From.*\n", file.read())[0]
        word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
        if word_2[0] in Spam_from:
            cheat_result[index] = 1
        index += 1
    
    eva_fun_2(label_test, cheat_result, 'NN')
    
    #--------------- end ---------------------
    
    
    
#     eva_fun_2(label_test, result, 'NN')
    f.write('------------------------------------\n')
f.close()

frequency is 1
dict length: 43861
Algorithm: NN
              precision    recall  f1-score   support

         ham       1.00      0.80      0.88      1299
        Spam       0.69      0.99      0.81       592

    accuracy                           0.86      1891
   macro avg       0.84      0.89      0.85      1891
weighted avg       0.90      0.86      0.86      1891

frequency is 2
dict length: 26435
Algorithm: NN
              precision    recall  f1-score   support

         ham       1.00      0.80      0.88      1299
        Spam       0.69      0.99      0.81       592

    accuracy                           0.86      1891
   macro avg       0.84      0.89      0.85      1891
weighted avg       0.90      0.86      0.86      1891

frequency is 3
dict length: 21068
Algorithm: NN
              precision    recall  f1-score   support

         ham       1.00      0.79      0.88      1299
        Spam       0.69      0.99      0.81       592

    accuracy                          

d:\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Algorithm: NN
              precision    recall  f1-score   support

         ham       0.96      0.72      0.82      1299
        Spam       0.60      0.93      0.73       592

    accuracy                           0.78      1891
   macro avg       0.78      0.82      0.77      1891
weighted avg       0.84      0.78      0.79      1891

frequency is 10000
dict length: 18


d:\python\python38\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:568: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Algorithm: NN
              precision    recall  f1-score   support

         ham       0.86      0.74      0.80      1299
        Spam       0.56      0.74      0.64       592

    accuracy                           0.74      1891
   macro avg       0.71      0.74      0.72      1891
weighted avg       0.77      0.74      0.75      1891



In [21]:
import time
 
localtime = time.asctime( time.localtime(time.time()) )
print(localtime)

Tue Jun  9 17:58:34 2020


In [22]:
# Remove common word in dictionary
for s__ in s:
    if s__ in dictionary:
        del dictionary[s__]
print(len(dictionary))

18


In [23]:
count = 0
for i in dictionary:
#     print(i)
    dictionary[i] = count
    count += 1
dictionary

{'com': 0,
 'p': 1,
 'http': 2,
 'www': 3,
 'face': 4,
 'width': 5,
 'b': 6,
 'size': 7,
 'font': 8,
 'nbsp': 9,
 'br': 10,
 'href': 11,
 'gif': 12,
 'src': 13,
 'tr': 14,
 'td': 15,
 'img': 16,
 'height': 17}

In [24]:
import numpy as np
train_ndarray = np.zeros((len(Training), len(dictionary)), dtype = int)
train_ndarray.shape

(4150, 18)

In [25]:
label_train = [0] * len(Training_Ham) + [1] * len(Training_Spam)
label_test = [0] * len(Testing_Ham) + [1] * len(Testing_Spam)

In [26]:
import pandas as pd
count = 0
for Ham_mail in Training_Ham:
    file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            train_ndarray[count][dictionary[w]] = 1
#     print(train_ndarray[count])
    count += 1
for Spam_mail in Training_Spam:
    file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            train_ndarray[count][dictionary[w]] = 1
    count += 1
train_dataframe = pd.DataFrame(train_ndarray)

In [27]:
import numpy as np
test_ndarray = np.zeros((len(Testing), len(dictionary)), dtype = int)
test_ndarray.shape

(1891, 18)

In [28]:
import pandas as pd
count = 0
for Ham_mail in Testing_Ham:
    file = codecs.open(Ham_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            test_ndarray[count][dictionary[w]] = 1
#     print(train_ndarray[count])
    count += 1
for Spam_mail in Testing_Spam:
    file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
    body = file.read().split("\n\n", 1)[1].lower()
    word = re.findall(r"[a-zA-Z']+", body)
    for w in word:
        if w in dictionary:
            test_ndarray[count][dictionary[w]] = 1
    count += 1
test_dataframe = pd.DataFrame(test_ndarray)

In [29]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(train_dataframe, label_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

In [30]:
result = neigh.predict(test_dataframe)

In [31]:
eva_fun_2(label_test, result, 'K-nn')

Algorithm: K-nn


ValueError: I/O operation on closed file.

In [ ]:
Spam_from = dict()
count = 0
for Spam_mail in Training_Spam:
    file = codecs.open(Spam_mail, 'r', encoding='utf-8', errors='ignore')
#     print(file.read())
    word = re.findall(r"From.*\n", file.read())[0]
#     print(Spam_mail)
    word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
    if not word_2 :
        continue
    Spam_from[word_2[0]] = Spam_from.get(i, 0)
#     print(word_2[0])
    

In [ ]:
Spam_from

In [ ]:
count = 0
cheat_result = result.copy()
for mail in Testing:
    file = codecs.open(mail, 'r', encoding='utf-8', errors='ignore')
#     print(file.read())
    word = re.findall(r"From.*\n", file.read())[0]
#     print(Spam_mail)
    word_2 = re.findall(r"[\w*\.?]*@[\w*\.?]*", word)
#     if word_2[0] == "admin@xent.com":
#         continue
#     if word_2[0] == "admin@lists.sourceforge.net":
#         continue
#     if word_2[0] == "admin@linux.ie":
#         continue
    if word_2[0] in Spam_from:
        cheat_result[count] = 1
        print(word_2[0])
#     print(word_2[0])
    count += 1

In [ ]:
cheat_result

In [ ]:
eva_fun_2(label_test, cheat_result, 'K-nn')

In [ ]:
for i in range(len(cheat_result)):
    if cheat_result[i] != result[i]:
        print(i)